In [1]:
import numpy as np
from dataclasses import dataclass
from numpy.typing import ArrayLike

@dataclass
class shape:
    """Class for keeping track of a shape."""
    pos_dict: dict

    def check_step(self, matrix, move) -> float:
        temp_arr = {}
        final_arr = {}
        move_list = symbol_dict[move]
        
        # This is so, so so ugly -> basically handling left / right separate from down
        for k, v in self.pos_dict.items():
            if v == '.':
                continue
            r = k[0]
            c = k[1] + move_list[1]
            
            # Runs into left / right wall or another item 
            if (c < 0) or (c > 6) or ((matrix[(r,c)] == '#') and (r >= 0)):
                temp_arr = self.pos_dict
                break
                
            # update if no break
            temp_arr[(r,c)] = v

        # And can we go downwards?
        for k, v in temp_arr.items():  
            if v == '.': # This burned me
                continue
            r = k[0] + move_list[0]
            c = k[1]
            
            # another value then we immediately stop (but col shift is possible so return temp_arr)
            if ((r > matrix.shape[0] - 1)) or ((r >= 0) and (matrix[(r,c)] == '#')):
                self.pos_dict = temp_arr
                return 1 # we can end it
            
            # update: 
            final_arr[(r,c)] = v
            
        # override current state
        self.pos_dict = final_arr
        
        # stop process if we hit bottom (really only matters for first step)
        if (self.find_max_row() >= matrix.shape[0] - 1):
            return 1
            
        return 0
    
    def find_max_row(self):
        return max([r[0] for r in self.pos_dict.keys()])

# directions
symbol_dict = {'>': [1, 1], '<': [1, -1]}

sh1 = {(0,c): '#' for c in range(2,6)}

sh2 = {(-2, 3): '#',
       (-1,2): '#', (-1, 3): '#', (-1,4): '#',
       (0, 3): '#'
      }

sh3 = {(-2,4): '#',
       (-1,4): '#',
       (0,2): '#', (0, 3): '#', (0,4): '#'
      }

sh4 = {(r,2): '#' for r in range(-3,1)}

sh5 = {(-1,2): '#', (-1, 3): '#', (0,2): '#', (0, 3): '#'}

shape_list = [sh1, sh2, sh3, sh4, sh5]

### Real

In [2]:
# Build our matrix and look at starting rocks
mat = np.zeros((1, 7), str)
mat[mat == ''] = '.'
# for sample
jet_stream = [x for x in '>>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>']
shape_count = 0
highest_element = 4
step = 0
runs = 2022

while shape_count < runs:

    # find lowest element, we start rows before that.
    if shape_count > 0:
        for i in range(mat.shape[0] - 1):
            x = np.argwhere(mat[i,:] == '#')
            if len(x) > 0:
                highest_element = i
                break

    # use to expand matrix: so ugly to handle the start
    boundary = (4 - highest_element)
    if shape_count == 0:
        boundary = 4
    if boundary > 0:
        mat = np.pad(mat, pad_width=((boundary,0),(0,0)), mode = 'constant', constant_values = '.')

    # build shape
    cur_shape = shape(shape_list[ shape_count % len(shape_list)])
    shape_count += 1

    while True: 
        #print(step)
        wind_dir = jet_stream[ step % len(jet_stream)]
        #print(wind_dir)
        step += 1
        out = cur_shape.check_step(mat, wind_dir)
        if out == 1:
            for k, v in cur_shape.pos_dict.items():
                if k[0] >= 0:
                    mat[(k)] = v
            break
            
    # find height of tower
    if shape_count > 0:
        for i in range(mat.shape[0] - 1):
            x = np.argwhere(mat[i,:] == '#')
            if len(x) > 0:
                highest_element = i
                break

print(f"Tower height: {mat.shape[0] - highest_element}")

Tower height: 3068


### Part 1:

In [3]:
# Build our matrix and look at starting rocks
jet_stream = [x for x in '>>><<<<><<><<<>>>><>>>><<><<<<>>>><<<<>><<>><<<>><<>>>><<<>>>><<<<>>><>><<<<>>><<<<>>>><<<>>>><<<>><<><><<>>><<<<>>>><<<><<<<>>><<<<>>>><><<<<><<<>>>><<<<>>><<><<<<>>>><<<>>>><<<><<<<>>><<<<>>><<<<>><<<<>>>><<><<<>><<<><<<>>>><<<<>><>><<><<<<><<<>>>><<<>><<<>>>><>><<<>>>><<>>><>>>><>>><<<<><<<>>><<<>>><<<<>>>><<<><<<<><>><<<>>>><>><>><>><>><<>><<<>>>><<><>><<<<>><>><<>><<<<>>><<<>><>>><>><<>>><>><<<<>><<<<><<<<>><<<>><<<>><<>><>>><<<<><<<>>><>>><>>>><<>><<<<><<<<>><<<>>>><<<<><<<>>><<<>>><<<>>>><<><<<>>>><>>><>>><<<<>>><<<<>>>><>>><<<<>><<>><<<<><><<<><>>><<<<>>><<><><<<<>>><<>>>><>>><>><<>>>><<>><<>>><>><<<><>>><<>>>><<><><<<>><<<<>><><<<<>><<>>>><<>>><>>><>>><>><<<>>>><<<<>><<><<<>><>>><<<>>><<<>>><<<<>>>><<<>>><<<<>>><<<><<>>>><<<<>>><<<<>><<<>>><<>>>><<<<><<<<>><<><<<<>>><<<<><<<><<<>><>>><<<>>><>><<<><<<<>><<<>>>><>>>><<<><<<>>><>>><<<>>><<<><>>><>><<<<>>><<>>><<>>>><><<<<>>><>><>>>><>>><<<<><<<>>><<>>>><>><<<>>>><<<>><><<<><<<>><<<><>>><><<<<>><<>>>><><<<>>>><>>>><<<>>>><<<<>>>><<<<>>><<<><<<>>>><>>>><<<<>><<<<>>>><<>><>><<<<>><<><<<>><<<<>>>><>><<<<><<<>><>>><<<<>><>>>><>>>><><<><<<<>><<<>><<>>><<<>>><<<<>>>><<<>>>><<<>><<<<>><<<<><>>><><<<<><<><<<><<>>>><<<><<><<>>>><<>>>><>>>><>><<<<>>>><<<<><>><<>>><><<<<>>>><<>><><<>>><<<<>>><<<>>><<<>><>>><<<<>>>><>>>><>>><<<>>>><<>>><>>>><<>>>><<<<><>>>><<>><<<><>>><<<>>><>>>><<<>>>><>><<<>>>><>><><<<<>>><<<<>><<<>>>><>>>><<<<>><<<>>>><<><<<><<<>>><<<<>>>><<<><<>><>>><<<<>>><<<>>>><>>>><<<<>>>><<>>><<<>><>>>><<<<>><<<<><>>><<<<>>><>>>><<<<>>><>>>><<><<>>>><<>>><<<<>>>><<<<>><<<<><<<>>><<<<><<<><>>>><>>><<>><<<<>><><>><<>>><>>>><><<<>>>><<>><<<>>>><<>><<><<>>>><<<<>>>><<<<>>><<<<>>><>><<<>>><<<>>><<<<>><<><>>>><<<>>><<>>>><<>>><<>>><<<<>>>><<<<>>>><>><<<>><<<<>>>><<<>>>><><><<<<>>><<<>>><<<<>>><<<><<<<>><<<>>><<<>><<>>>><<>>><<>>>><<<><<>>>><<><<<><<<>>><<<<>>><>>><<<<>>><>>><<<>>>><>>><<<><<><>>><<>>><<>>>><<<<>><<<<><><<<<>>>><<<<>><<<><<<>><<<>>>><<>><<>>>><<><<>>><<<<><<<>><<<>>><>><<>><<<<>>>><<>>>><<<><>>><<<><<>>><<<<>>>><<<>>><<>>>><<<<>>>><<<>>>><<<>><<<>><<<<><<<<><<>>><<<<>><<<>>><>><>><><<<>>>><<>>><>>><<><>>>><<>>><<>>><<><><<<<>><<<>><<<<>>>><>>>><>>><<<<>><<<><<<>>><<<<><<><>>>><<<>>><<<><<>>>><<<>><<<<>>><>><<<<>><<<>><><<<<><<<<>><>><<<>>>><<<>>><<<>>><>>><<<>><<>>>><<<>>><<<<>><>>><<>>>><>><<<<>>>><>><<<>>>><>>><<<<>>><><<<>><<<>>><>><<<<><<<><<<>>>><>>>><<<<><>>>><<>>>><>>><<>>><>>><<<<>>><<<><>>>><>><<>>><<<<>><<<><<<><>>>><<<<>><<>><<<<>>>><>>>><><<<<>><<<>>>><<<>>>><<<>>>><<>>>><<<><>>>><<<>>>><<<<><<<>><<<<><><<<>><<>><<>><<><<<<><<>><<<<>>>><<<>>><<<><>><<>>>><<>>><>><<><<<><>>><<><>><<<><>><<<><<<>><<>>><<<>>>><<<<>><<<>>><>>><<<<>>><><<<<>>><<<<>>>><>><<>>>><<<<>>><<<>>><><>>><><<<<>>>><<<>>>><<<<>><<><<<<>>>><<><>>><<<>>><<>>><<<><<>>>><<<<>>><<<>>>><<<<>><<<>>>><<<>><<>><><<<>><<<<>>>><>><<<<>><<><<<<>>>><>><<>>>><>>>><><<<<>>>><<>><>>>><<>>>><<><<<<><>>>><>>><<>>><<<><<>>><<>>>><>>>><>>><>>>><<<>><<<<>>><<>>><<<<>>><><<>><<>><<<>><<>><<<>>>><<>>>><<<<>>><<<>>>><<<>>>><><<<<>><<>><<<<><<<>>><<<>><<<<>><<<<>>>><<<<>><<<>><<>>>><<<<>>>><<<<>>><<<<>>>><<>>><<>>>><<>><<<>>><<>><><<<>><>>>><<<>>><>>><<<>>>><<<<><>><<<<>><<>>>><<>><<<<>>>><<<><<><>>>><>>>><<><<<>>>><<>><<<<>>>><<<<><>><<<>><>>><<<<><>>>><<><<>>>><<<<>>>><<<<><<<>><<>><<>><>>><<<><<<<>>>><<>><<>>>><>>>><<<><>>>><<<<>>>><<<<>>>><<>>><<<<>>><<<>><>>>><<<><<>>><<>>>><><<<>><>>>><<<><<>><<>><<<>>><>><>><<>>>><>>><<><<>><<<><>>>><<<<>>><<<>><<<><<<<>><<<><>><<<><<>>><<<>>><>><>><<<>>>><<<><>>>><>>><<>>>><<<>><<<<>>>><<<>><>>>><<><<<>><>>><<<>>><<>>>><><>><<<>>>><<<><>>>><<<>>><<<>><<>>><>><>>>><<<<>>>><<><><<<<>>><>><<<<><>><<<>>>><<<<>>><<<<>>><<>><><<<<>><<<<>>>><<<<>>><<<>>>><<<<>>><<<<>>><<<>><>><<>>><<<>><>>><<><<<>>><>>><<<><<>>><>><<<<><<<>>><>><<<<>>><><>>>><<>>>><<<>><<<>>>><<<<><<>><>><>>>><><<<<><<><<><<<>>><><<<<>><<>><<<>>>><<>>><<<>><>><>>>><<>>>><>>>><>><>>>><>>><>><<<<>><<<<>>>><<<><<<><<<<>><>>><<>><<<<>><<<<>>><<<<>>><<<>>>><<<<>><<<>><<<><<<><<<<>>>><<<><<>>><<<>>><<><<<<>><<>>><><<<<>><><<<>>>><<<>>>><<<<>>>><><<>>><><><<>><>><>><<<>>><>><<<>><<<<><<<<>>>><<<>>><>>><>>>><<><<>>>><>>><><>><<>>>><<<>><<<<>><>>>><>>><<<<>>><<>>>><><<<>>><>>><>>><<<<>>>><<>><<<>>>><<>><>>>><<>><<>>><<<<>>>><>>><>>>><<<<>>><<<<>><>>>><<<>><><<<<>>>><<>>><<><<><><<<>>>><<>>>><<>>><>>><<<>>><>>><<<>>>><<<>><<<<>><<>>>><<><<<><<<>><<<<>>>><<<>><<>>><<<<>>><<<>>><<<<><<><>>>><<<<>><><<><<<>><<<<><<<>><<<>>><<>><<<<>>><<<><<<<>><<<<>>><><<<>>><<<<>>><<>>><>>>><<>>><<>><<<><<<<>><<>>>><<<<>>><<<<>>>><<<>><<<<>>>><<<<>>><<<>>><>><<><<<<><<<<>>>><<<><<><<<<><<<<>><>><<<>><<><<<><<><<<>><<<<><>>>><<<>>>><>><<>>>><>>>><>>>><<<>><<<>><<<<>>>><>>>><<<>>><<>>>><<<>>><<>>>><<<<>>>><><>>><><<>>><<<<>>>><>>>><<<>>>><<>>><<>><<<>>><<<<>><<<<>><<>><<<>><<>>>><>>><><<<<><><>>>><<<>>>><<<<>>><<>><>><<<<><>>><<<<>>>><<<<>><>>><<<>>><>>>><><>><>>><><>>>><><>>><<>><<><<<<>><<<<><<<<><<>><<<<><<<>><<<>>>><><<<<>>><<><<<<>>>><>>>><>>><<<>>><<<>>>><<>>><<><>>><<<><>><<<<><<<<><><<<>><><<>>>><<<>><<><<<<>><<>>><>><<>>><>><<<><>>><<>>>><<<<>><>>>><<<>>><<<>>><<>>>><<<>>><<<><>><<>>>><<>><>><<<<>><<<>>>><><<<<>>><>>><<>>><<<<>><<<>>><<>>>><<<>>>><><<<>><<>>>><<<>>><<<<><<<<>><<<<>><<<<>><<>>><<<>><<<>>>><<<<>>><<>>><<<<>>><<<>><<<<><><>><<>>>><<>>><<<<><<<><<<<><<<><><<<>>><>>>><<<<>>>><<><<<>>>><<><<<><><<>><<<<>>><<<><<<>><<<>>><>>><<<<>>><>><<>>>><<<<><<<>><<><>>><<<<>>>><><<>><>>>><<<>>>><<<<>>>><<<>>>><>>><<<>>><<<<><<<<>>>><<<<>>><<><<<>><<<<>><<<>>>><<<<>><<>>><>>><<<<><><<<>><>><>><<<>><<<<>>>><<>><>><<<>>><><<<<>><<>><<<><<>>>><><<<><<>><>><<>>>><<<<>><>>><<<<><<<><<>><<<<>><<<<>>>><<>>><<<<>>><<<>>>><<>>><<<><>>>><<<<>><<<<>><<>>>><<<>>>><<>><<<>>><>><<<<>>>><>><<<<><<<>><<<<>><>><>>>><<<>>>><><><>>>><<<<>>><<<>>><<<<><<>><<><<>>>><<<><<<<>>>><<>>>><<>>><<>><<<<>><<<<>>><>>><>>>><>><<<>>>><<>><<<><<<<><<<>>><<<>>>><<<><<>><<>>><><<<<>><<><<<<><<<>>>><<<<>><<>><<><<<>>><<<<>><<>>><<<>>><<<>><<<>>>><>>>><<>>><<<<>>><>>>><<<><<<>>><<<>><>>><>><<<>>><>>><<<>>><<<<><<<><<<<>><<<<>><<>>><>>><<<<><<>>>><<<<><<>>>><>>>><>>><<>>>><<>>>><<<<><<>><<>><<><>><><<<<>>><>>><<>><<<<>>>><<<<>><>><<<<>><<<>>><<<<>><<>><<>>><<<>>><<<<>><<<<><>><<<<>>><>>>><>><<>>>><>>>><<<<>><<>>><<>>><<>>>><><<><<<<>><<<<><<>>><>>><<<<><<<<>>><<<<>><<<>><<>>>><<<<><<>>>><<<>>>><<<<>>>><<<<><<<<>><<<<>>>><<<>>>><>><>>><<<<>><<<>><<<>><<>><<<<><>><<>><><<<<><<<<>>><<><<<<>><<<>>>><<<><<<<><<>>>><>>><>>><<<>><<<>>>><>>>><<>>><<<<>>><<>><><<<>>>><<><>>>><>><>>>><<<>>>><>>><<>>><>><>>><<<<><<>><<>><>>><>>><<<>><<><<>><><<>>><<>><<>><<<>><<<>>>><>>>><><<<<>>>><<<><<<>>><>><<<<><<<<>>><>>>><<>>>><<>>><>>>><>>>><<<<>>>><<>>><<<<>>><<<>><>>><<<<>>><>><<>>>><<>><<>><><><<<>>>><><<<>>><<<>><<>>>><<>>>><<>>><<>>><<>>><<<><<<<>><<<<>><<<<>><<><<<<>>>><<><<<<>>>><<<<>><<<>>>><<<>><<<<><<<<>>>><<<>><>>>><><>>>><<<<>><<<>>>><<<><<<>>>><<<<><<<>><><<>>><<<<><<>><<<<>>><<<<><<<<>><<<<><<><<<<>><><<<>>><>>>><<<<>><<<>>><<>><<<><>>>><>>><<<<>><>>>><<<><<>>>><<<<>>><<<>>><<><<<<>>>><<<>>><<>>><>>>><<<>>>><>>><<<<><>>>><<>>><<<><>>>><>>>><<<><<><<>><<<<><>>><><<<>>><<>>>><<<<>>><>><<<>><<<>>><<>>><<>>><<<>><<<<>><<<>>><<<>><<<><<<<>>><>>><<<<><<<><<<>>><>>><<><<<<>><<<<><<>><<>>>><<<><>>>><<>>>><<<><>><><<<<><<<><>>><<<<>><>><<<><<><<<<>>><>>><<<>><<<>><>>><<><<<<>>>><<>>>><><><>>>><<>>><<>>><<<<>><<><<<<>><>>><><<<<>>><<<<><<<<>>><<<>>><<<<>>>><<>>>><<<<>><<<>>>><<>>><<<<>><<><<<<><<<><<<><<<<>>><<<<>>>><<>>><<<<>><>><<>><<>>>><>>><<<<><<<<>>><<<>><<><<>>><>>><<<><>>><>>><><>><<<>>>><<<<>>><>><<<><<<>>>><<><<<<>>><<<>>>><<<><<>>>><<<<>>><<><<>>><<>><<>>>><<>>>><<<>>><<<<>><<>><<<<><<<>>><>>>><<>>><>>><<>>><<<>>>><>>>><>>>><<<><>>><<<<>><<<>>><<<>>>><<<<>>>><>>><<<<>>>><<<>>>><>>><<>><<<>>>><<<>><<<>>>><><><>>><<><>>><>>>><>>>><<<<>>><<<><<>>>><<<>>>><<<><>><>>><<<>>>><>><<>><>><<<>>>><<<<>>><<<<><<<<>>>><<<<>>><<>>>><<<<>>>><<<<>><<<>><><<<>>><<<<>>><>>>><<<<>><<>><>><<>>><>>>><<<<><<<><<>>><<<>><<<<>>><<>>><<<>>><<<<><<<>>>><<<>>>><<<>>><>>>><<>>>><<>><<><<>>><><<>><<<<>>>><<<><>>>><<<>><<<<>>>><>>><<<><>>><<<<>>><><<>>>><<<<><<<<>><>>><<<><>>>><<<<>><>>>><<<><<<>>>><<<>>><<<><<<>><<<<>>>><<<>><<<<><<<><<><<<<>>><<<<>>>><<><<<<>>>><><<<<>>>><<<>>><<<>>><<<<><>><<<<>><<<><<<<>>><<><<<<><<>>><<<><<<<><>>><<<<><<<>>>><<<<>>>><<>>>><>><<<<>>><><<<<>>>><<<<>><<<>><<<><<>>><<>>>><>>><<>><<<>><<>>><>>>><<>>>><>>><<<<>><><>>>><<><<<<>>>><<<<>>>><<>><>>><<<>>><<>>>><>>>><>>>><<<<>><<<<>><<<<><<<>>>><>>>><<<<>>>><>><<>>>><<<<>>><<<>>><>><<><<<>>>><>>><<>><<<><>><<<<>><<><>>><<<<><<<>><>>><<<>>>><<>>>><<<<>>><<>><>>><<<<>><<>><<<<>>><><<>>><<><<<<><<<<>><>>><<><<<><<<<>><>>><<>>>><<<>><<<>><<<>>><<<>>>><>>>><<>>><<<>>><>><>>>><>>>><<<>><>>><<<>><<>><>><<<<>>><>>><<<><<<>>>><<<>><<<>>><>>><<<><>>>><<<<><>><<>><<<<><<<>><<>>><<<><>>><<<><<<<><<<<><>><<>>>><<<<>>><<<<>><<<<>>><<<<><<><<<>>><><<>><<>>>><<>>><><<<<><<<<>>><><<<>>>><<<<>>>><<<<>>>><<>><<>><<>><<<>><>>><>><<<<>><<><<<>>><>>><><>>><<<>>>><<>>><>><<<>>>><<<>><<><><<<><>><<<<>>><<<<>><<<<>><<<>>><<<<>>><<<<>>>><<<>>><<<<>><<<<><><<<>>><<<><<><<><<<>><<>>><<>><<><<>>>><>><<<<>>>><>>>><<<<>>><<<<>>><>><<<><<<>>>><<<<>><<<<><<><<<<>>>><<<<>><<<<>>>><<<<>><<<>><<>><<<<>><>><<>>>><<<<><<<><>>><>><<<>>>><><<<><>>>><<<>>><<<>><<<><<<>><<><>>>><<><>>><>><<<>>><<<<>><<><<>><<<<>><<<<><<<>>>><<<<>><<<>>>><>>><<<>><<>>>><<><><<<<><<<<>>>><<<>><<<<>><>>>><<><><<>>>><<<><<<<>><>><>>><<><<<<>>>><<>><<<<><<>>><<<>>><<<>>><<<>>>><>><<<>>>><<<>>>><><<<<>>>><>><<<<>><<<<>>>><<>>><<<>>>><<>><<><<<>>><<>>>><<<<>><<<<>>><>>>><<<>>><<>>>><<>>><><>><<<<>><<<>>>><<>><<<>><<<>>><<<>><>><><<<<>>><<<><<>>>><<>>>><<>><<<><<<><<<<><<><>>>><<<>>>><<<<>>><>><<>>>><>><<<>>>><<<>><<>>>><<<<>>><<<><><<>>><<>><<<>>><<<<>><<>>>><<<><<<<><<>>>><<>>><<<<><<<<><<<>>><<>>><>><<><<><<>>><><>>>><<<>><<>>>><>>>><<<<>><><<<<>><<<<>>>><<>>>><<<<><<<>>><<>>><<><<<<>><>><<><<><<><<<><<<<><>><<<>>>><<<<>>>><<<><>>>><<>>><><>>><<<>>><<<><<>>><<><><<><<<<><<<<>>>><><><<<<>>><>><<>>>><<<<>>>><<<>>><<<><<><<<<><<<<>>><<<<>>>><<<>><<<<>>>><<><<<>><><>>>><<<>>>><<<>>>><<><<<>><>>>><<<><>><<<<><<<<>>><<<<>>>><<><<<>>>><><><<<<><<<<><>><<><>>>><<<>><<>><']

# Build our matrix and look at starting rocks
mat = np.zeros((1, 7), str)
mat[mat == ''] = '.'

shape_count = 0
highest_element = 4
step = 0
runs = 2022

while shape_count < runs:

    # find lowest element, we start rows before that.
    if shape_count > 0:
        for i in range(mat.shape[0] - 1):
            x = np.argwhere(mat[i,:] == '#')
            if len(x) > 0:
                highest_element = i
                break

    # use to expand matrix: so ugly to handle the start
    boundary = (4 - highest_element)
    if shape_count == 0:
        boundary = 4
    if boundary > 0:
        mat = np.pad(mat, pad_width=((boundary,0),(0,0)), mode = 'constant', constant_values = '.')

    # build shape
    cur_shape = shape(shape_list[ shape_count % len(shape_list)])
    shape_count += 1

    while True: 
        #print(step)
        wind_dir = jet_stream[ step % len(jet_stream)]
        #print(wind_dir)
        step += 1
        out = cur_shape.check_step(mat, wind_dir)
        if out == 1:
            for k, v in cur_shape.pos_dict.items():
                if k[0] >= 0:
                    mat[(k)] = v
            break
            
    # find height of tower
    if shape_count > 0:
        for i in range(mat.shape[0] - 1):
            x = np.argwhere(mat[i,:] == '#')
            if len(x) > 0:
                highest_element = i
                break

print(f"Tower height: {mat.shape[0] - highest_element}")

Tower height: 3085


### Part 2:

This should just require finding a pattern and figuring how many shapes it takes for that pattern to occur. 

I think I need to use a window here & store info on max tower height each step
- Iterate over rows 1 - 10 for examples for the first 3 shapes dropped:
- Look over rows 11 - 20, are these the same value? If not expand 1 - 13 (assume next shape takes 3), compare to 14 - 26...